In [ ]:
import pandas as pd
import numpy as np
import holoviews as hv
# import geoviews as gv
# import geopandas as gpd
from holoviews import opts, dim
import panel as pn
import panel.widgets as pnw
pn.extension()
hv.extension('bokeh')

In [ ]:
df0 = pd.read_csv('regions_fadn.csv')

In [ ]:
# europe = gpd.read_file('fadn_regions.shp')

In [ ]:
choose = ['YEAR','REGION','TF14']
cols_bal = ['SE446','SE450','SE455','SE460','SE470','SE475','SE480']
cols_inc = ['SE135', 'SE206', 'SE256', 'SE281', 'SE336', 'SE600','SE360', 'SE370', 'SE375', 'SE380', 'SE420']
cols_com = ['SE011','SE025','SE080','SE131','SE132','SE135','SE136','SE206','SE207','SE256',
            'SE270','SE360','SE441','SE455','SE420','SE605']
oles = list(set(choose + cols_bal + cols_inc + cols_com + ['FADN_2012_']))
negs = ['SE281', 'SE336', 'SE275', 'SE370', 'SE360', 'SE375', 'SE380', 'SE365']
names = {'SE011': 'Tot.Hours (hrs)', 'SE025': 'Tot. Area (ha)', 'SE080': 'Liv.Units (LU)', 'SE131': 'Tot.Out. (€)',
         'SE132': 'T.Out/T.Costs (ratio)', 'SE135': 'Crop.Out (€)', 'SE136': 'Cr.Out/Ha (€/ha)', 'SE206': 'Liv.Out (€)',
         'SE207': 'Lv.Out/L.Uni (€/LU)', 'SE256': 'Oth.Out (€)', 'SE270': 'Tot.Costs (€)', 'SE281': 'Tot.Spec.Costs (€)',
         'SE336': 'Tot.Farm.Overheads (€)', 'SE360': 'Depreciation (€)', 'SE370': 'Wages (€)', 'SE375': 'Rent (€)', 'SE380': 'Interest (€)',
         'SE420': 'Family Inc. (€)', 'SE436': 'Tot.Asse. (€)', 'SE441': 'Fixed Assets (€)', 'SE446': 'Land etc (€)',
         'SE450': 'Buildings (€)', 'SE455': 'Machinery (€)', 'SE460': 'Br.liv (€)', 'SE465': 'Tot.cur.ass (€)',
         'SE470': 'Non-br.liv (€)', 'SE475': 'Prod. stock', 'SE480': 'Oth. circul. (€)',
         'SE600': 'Bal.Subs&Taxes (€)', 'SE605': 'Subsidies (€)'}
names_rvs = {v: k for k, v in names.items()}
colors_out = ['#C70039', '#66DDF0']
colors_in = ['#66DDF0', '#C70039' ]
inputs = ['Tot.Hours (hrs)', 'Tot. Area (ha)', 'Tot.Costs (€)', 'Depreciation (€)', 'Tot.Asse. (€)', 'Machinery (€)']
outputs = ['T.Out/T.Costs (ratio)', 'Crop.Out (€)', 'Cr.Out/Ha (€/ha)', 'Liv.Out (€)', 'Lv.Out/L.Uni (€/LU)', 'Family Inc. (€)', 'Subsidies (€)']
ranks= {}
sygr = {}


In [ ]:
year1 = pnw.Select(name='Year', options=list(df0.YEAR.unique()))
perif1  = pnw.Select(name='Region', options=sorted(list(df0.REGION.unique())))
tf1  = pnw.Select(name='TF14', options=sorted(list(df0.TF14.unique())))
met1 = pnw.Select(name='Map Variable', options=names_rvs)

@pn.depends(perif1.param.value, tf1.param.value, year1.param.value, met1.param.value)
def assets1(perif,tf,year,met):
    df = df0
    bal = df[choose + cols_bal]
    inc = df[choose + cols_inc]
    com = df[choose + cols_com]
    counter = df.YEAR[(df['REGION'] == perif) & (df['TF14'] == tf) & (df['YEAR'] == int(year))].count()
    if counter == 1:
        reg = com[(com['REGION'] == perif) & (com['TF14'] == tf) & (com['YEAR'] == int(year))]
        frame = com[(com['TF14'] == tf) & (com['YEAR'] == int(year)) & (com['REGION'] != 'All')]
        for var in cols_com:
            reg_val = reg[var].values[0]
            ranked_list = list(frame[var])
            ranked_list.append(reg_val)
            ranked_list = sorted(ranked_list)
            ranks[names[var]] = ranked_list.index(reg_val) + 1
            mesos, typikh = frame[var].mean(), frame[var].std()
            if typikh !=0:
                sygr[names[var]] = (reg_val - mesos) / typikh
            else:
                sygr[names[var]] = 0
        scatter0 = hv.Scatter((sygr))
        spikes = hv.Spikes(scatter0)
        eis = spikes[inputs].redim.range(y=(-2,2)) * scatter0[inputs]
        eis.opts(
            opts.Scatter(height=300, width=350, size=20, fill_color='#949194', tools=['hover']),
            opts.Spikes(color=dim('y').norm(), cmap=colors_in, line_width=20, invert_axes=True))
        eja =  spikes[outputs].redim.range(y=(-2,2)) * scatter0[outputs]
        eja.opts(
            opts.Scatter(height=300, width=350,size=20, fill_color='#949194', tools=['hover']),
            opts.Spikes(color=dim('y').norm(), cmap=colors_out, line_width=20, invert_axes=True))
        pre_lay = eis.relabel('Distance from mean of Regions').redim.label(y="Std's", x='Variable') + \
              eja.relabel("with same TF in Year").redim.label(y="Std's", x='Variable')
        pre_lay.opts(shared_axes=False, toolbar='right')
        synolo = frame['SE131'].count()
        
        reg_in = int(reg['SE270'].values[0])
        reg_out = int(reg['SE131'].values[0])
        reg_fam = int(reg['SE420'].values[0])
        scatter1 = hv.Scatter(frame, 'SE270', ['SE131', 'REGION'])
        line1 = hv.Slope.from_scatter(scatter1)
        point1 = hv.Points((reg_in, reg_out))
        hline1 = hv.HLine(np.mean(frame['SE131']))
        vline1 = hv.VLine(np.mean(frame['SE270']))
        regr1 = (scatter1 * line1 * point1 * hline1 * vline1).relabel('Total Cost vs Total Output').opts(shared_axes=False)   
        regr1.opts(opts.Scatter(width=600, height=250, tools=['hover'], size=5),
                   opts.Slope(line_color='r', line_dash='dashed'),
                   opts.Points(color='r', size=10, marker='^'),
                   opts.HLine(color='orange', line_dash='dotted'),
                   opts.VLine(line_color='orange', line_dash='dotted')
                  ) 
        scatter2 = hv.Scatter(frame, 'SE270', ['SE420', 'REGION'])
        line2 = hv.Slope.from_scatter(scatter2)
        point2 = hv.Points((reg_in, reg_fam))
        hline2 = hv.HLine(np.mean(frame['SE420']))
        vline2 = hv.VLine(np.mean(frame['SE270']))
        regr2 = (scatter2 * line2 * point2 * hline2 * vline2).relabel('Total Cost vs Family Income').opts(shared_axes=False)
        regr2.opts(opts.Scatter(width=600, height=250, tools=['hover'], size=5),
                   opts.Slope(line_color='r', line_dash='dashed'),
                   opts.Points(color='r', size=10, marker='^'),
                   opts.HLine(color='orange', line_dash='dotted'),
                   opts.VLine(line_color='orange', line_dash='dotted')
                  )    
        reg_xrono = com[(com['TF14'] == tf) & (com['REGION'] == perif)]
        curve1 = hv.Curve(reg_xrono, 'YEAR', 'SE270', label='Costs')
        curve2 = hv.Curve(reg_xrono, 'YEAR', 'SE420', label='Family')
        curve3 = hv.Curve(reg_xrono, 'YEAR', 'SE131', label='Output')
        curve4 = hv.Curve(reg_xrono, 'YEAR', 'SE605', label='Subs')
        curves = (curve1 * curve2 * curve3 * curve4).opts(opts.Curve(tools=['hover']))
        curves.opts(width=700, height=280, legend_position='right', ylabel='', title='Time Series')
        inc = inc[(inc['REGION'] == perif) & (inc['TF14'] == tf) & (inc['YEAR'] == int(year))]
        inco = inc.to_dict('records')
        inco = inco[0]
        del inco['YEAR']       
        del inco['REGION']
        del inco['TF14']        
        evi = {}
        x_ticks = []
        etik = []
        x_0 = 0.1
        x_1 = 0.9
        y_0 = 0
        y_1 = 0
        for var in inco:
            if inco[var] == 0:
                inco[var] = 0.1
            if var in negs:
                inco[var] *= -1
            evi[var] = (x_0,y_0,x_1,y_1+inco[var])
            x_0 += 0.4
            x_ticks.append((x_0, names[var]))
            if var != 'SE420':
                etik.append((x_0, (evi[var][1]+evi[var][3])/2, inco[var]))
            else:
                etik.append((x_0, inco[var]/2, inco[var]))
            x_0 += 0.6        
            x_1 += 1
            y_0 = evi[var][3]
            y_1 += inco[var]       
        evi['SE420'] = (10.1, 0, 10.9, evi['SE380'][3])
        lista = list(evi.values())        
        color_exp = (dim('y0') < dim('y1')).categorize({True: '#0DDA9F', False: '#CA2921'})
        rect = hv.Rectangles(lista).opts(
            title='Family Income Calculation', labelled=[], width=600, height=300, color=color_exp, xticks=x_ticks, xrotation=45, tools=['hover'])
        labels = hv.Labels(etik).opts(text_font_size='10pt')
        eisodhma = rect * labels * rect[10:11].opts(fill_color='#02C2F5',color='#02C2F5', alpha=0.5)
        eisodhma.opts(width=700)
        bars = hv.Bars(ranks, label='Ranking').opts(invert_axes=True, tools=['hover'], axiswise=True, height=300, width=700, bgcolor='#CCDED5').redim.range(y=(0,synolo)).redim.label(y='Rank', x='Variable')
        
#         df = df[(df['TF14'] == tf) & (df['YEAR'] == int(year))]
#         comp = europe.merge(df, on='FADN_2012_', how='outer')
#         xarths = gv.Polygons(comp, vdims=[met, 'FIRST_FADN', 'CNTR_ID']).opts(
#             tools=['hover'], width=800, height=700, color_index=met, colorbar=True, toolbar='right', cmap='coolwarm')

        return pn.Tabs(('various',pn.Row(pn.Column(pn.Row('No of Regions',synolo),bars,pre_lay,eisodhma),
                                         pn.Column(curves,regr1,regr2))),('map','cartopy fails to load'))
        
    else:
        return 'No data available'



widgets1 = pn.Row("<br>\n###Select Region/Type of Crop/Year", perif1, tf1, year1, met1, width=1400, background='#CCDED5')
pn.Column(widgets1, assets1).servable()